In [8]:
path = 'E:\\Notebook\\Digit Recognizer\\'

In [16]:
#librry cell
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


In [10]:
#read data
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')
y_train = train['label']
X_train = train.drop(labels = 'label', axis = 1)

In [11]:
#normalization
X_train = X_train/255.0
test = test/255.0

In [12]:
# Reshape in 3-ds (height = 28px, width = 28px , canal = 1)
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [18]:
#encoding
Y_train = to_categorical(y_train,num_classes = 10)

In [19]:
#split train and validation
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.2, random_state = 2)

In [21]:
#CNN sequential model
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = 'Same', activation = 'relu', input_shape = (28,28,1)))

model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = 'Same', activation = 'relu'))

model.add(MaxPool2D(pool_size =(2,2)))

model.add(Dropout(0.25))

##2nd set of layers

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation = 'relu'))

model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))


In [22]:
#optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics = ["accuracy"])

In [23]:
#learning
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience = 3, verbose = 1, factor = 0.5, min_lr = 0.00001)
epochs = 5 
batch_size = 86

In [24]:
#argumentation
datagen = ImageDataGenerator(featurewise_center=False,  samplewise_center=False,featurewise_std_normalization=False, 
        samplewise_std_normalization=False, zca_whitening=False, rotation_range=10,zoom_range = 0.1,width_shift_range=0.1,
        height_shift_range=0.1,horizontal_flip=False,vertical_flip=False) 

datagen.fit(X_train)

In [25]:
#fit model
history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),epochs = epochs, validation_data = (X_val,Y_val),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size, callbacks=[learning_rate_reduction])

Epoch 1/5
 - 137s - loss: 0.4598 - accuracy: 0.8502 - val_loss: 0.0770 - val_accuracy: 0.9767


C:\Users\shrey\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 2/5
 - 130s - loss: 0.1448 - accuracy: 0.9559 - val_loss: 0.0507 - val_accuracy: 0.9844
Epoch 3/5
 - 141s - loss: 0.1044 - accuracy: 0.9693 - val_loss: 0.0643 - val_accuracy: 0.9811
Epoch 4/5
 - 144s - loss: 0.0873 - accuracy: 0.9744 - val_loss: 0.0576 - val_accuracy: 0.9849
Epoch 5/5
 - 145s - loss: 0.0778 - accuracy: 0.9761 - val_loss: 0.0319 - val_accuracy: 0.9906
